In [1]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo

!pip install gdown

!gdown 1hKQDX7SZw5AzysFh32gr0qxTOHLzAQIC

Downloading...
From: https://drive.google.com/uc?id=1hKQDX7SZw5AzysFh32gr0qxTOHLzAQIC
To: /kaggle/working/SemEval2025_task11_processed.zip
100%|██████████████████████████████████████| 2.99M/2.99M [00:00<00:00, 5.24MB/s]


In [2]:
!rm -rf processed_data

!unzip -q SemEval2025_task11_processed.zip

In [3]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

In [4]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [5]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [6]:
# models_list_1=[

#              "funnel-transformer/medium",

#              "google/electra-base-discriminator",

#              "facebook/bart-base",

#         ]



# models_name_1=[

# "funnel-transformer-medium",

# 'electra-base',

# "bart-base",

# ]



# models_list_2=[

#              "facebook/bart-large",

#               "FacebookAI/xlm-roberta-base",

#           ]



# models_name_2=[

# "bart-large",

# "XLM-Roberta-base",

# ]



# models_list_3=[

#       "FacebookAI/xlm-roberta-large",

#      "google-bert/bert-base-multilingual-uncased",

#           ]



# models_name_3=[

# "XLM-Roberta-large",

# "bert",

# ]



# models_list_4=[



#      "distilbert/distilroberta-base",

#      "distilbert/distilbert-base-uncased",

#      "distilbert/distilbert-base-multilingual-cased",

#       ]





# models_name_4=[

# "distil-roberta-base",

# "distil-bert",

# "distil-bert multilingual",

# ]



# models_list_5=[

#      "microsoft/mdeberta-v3-base",

#      "albert/albert-base-v2",

#      "xlnet/xlnet-base-cased",

#      "google-t5/t5-base",

#       ]

# models_name_5=[

# "DeBERTAV3",

# "albert-base",

# "xlnet-base",

# "t5",

# ]



models_list_1=[



    # "facebook/bart-large",



    # "FacebookAI/xlm-roberta-base",



    # "microsoft/mdeberta-v3-base",



    # "xlnet/xlnet-base-cased",



    # "google-t5/t5-base",



    # "albert/albert-base-v2",

    # 'microsoft/Phi-3.5-mini-instruct',

    # "google/gemma-2-2b-it",
    "unsloth/Llama-3.2-1B-Instruct",

    # "openai-community/gpt2",

    # "openai-community/gpt2-xl",

    # "openai-community/gpt2-large",

    # "openai-community/gpt2-medium",

    ]















models_name_1=[



# "bart-large",



# "XLM-Roberta-base",



# "DeBERTAV3",



# "xlnet-base",



# "t5",



# "albert-base",

  # 'microsoft/Phi-3.5-mini-instruct',

    "google/gemma-2-2b-it",

#  "openai-community/gpt2",

    # "openai-community/gpt2-xl",

    # "openai-community/gpt2-large",

    # "openai-community/gpt2-medium",



]

In [7]:
kind="train"

task="track_a"

langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]

# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [8]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"

    if not os.path.isfile(processed_path):

      print(processed_path)

      continue

    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




processed_data/train/track_a/sum.csv


In [9]:
train_data=all_csv.drop(columns={"unnamed: 0"})

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,clean_message,surprise
0,afr_train_track_a_00001,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,afr_train_track_a_00002,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,afr_train_track_a_00003,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,afr_train_track_a_00004,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,afr_train_track_a_00005,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,tir_train_track_a_03678,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,tir_train_track_a_03679,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,tir_train_track_a_03680,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [10]:
train_data = train_data[

    train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,clean_message,surprise
0,0,afr_train_track_a_00001,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,1,afr_train_track_a_00002,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,2,afr_train_track_a_00003,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,3,afr_train_track_a_00004,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,4,afr_train_track_a_00005,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...,...
25557,25557,tir_train_track_a_03677,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,25558,tir_train_track_a_03678,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,25559,tir_train_track_a_03679,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,25560,tir_train_track_a_03680,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [11]:
#update label column name

label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']



y = train_data[label_columns].values



mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=1)

train_indices, valid_indices = next(mskf.split(train_data, y))



train = train_data.iloc[train_indices].reset_index(drop=True)

valid = train_data.iloc[valid_indices].reset_index(drop=True)



x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()



y_train_df = pd.DataFrame(y_train, columns=label_columns)



label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']

print(label_distribution)

          count_0  count_1
anger       16358     4092
fear        17983     2467
joy         16022     4428
sadness     16960     3490
surprise    18594     1856


In [12]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [13]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item



    def __len__(self):

        return len(self.labels)

In [14]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    # BATCH_SIZE=1

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [15]:
def compute_metrics(p):

  predicted_logits = p.predictions

  print(f"Type of predicted_logits: {type(predicted_logits)}")

  if isinstance(predicted_logits, tuple):

        predicted_logits = predicted_logits[0]

  if not isinstance(predicted_logits, np.ndarray):

        predicted_logits = np.array(predicted_logits)

  if len(predicted_logits.shape) == 3:

        predicted_logits = predicted_logits[:, 0, :]

  predicted_probabilities = expit(predicted_logits)

  threshold = 0.5

  predicted_classes = (predicted_probabilities >= threshold).astype(int)

  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average="weighted")

  accuracy = accuracy_score(y_valid, predicted_classes)

  return {

        "accuracy": accuracy,

        "precision": precision,

        "recall": recall,

        "f1_score": f1

    }

In [ ]:
def evaluate_func(trainer):

  trainer.evaluate()

  predictions = trainer.predict(valid_dataset)

  predicted_logits = predictions.predictions

  print(f"Type of predicted_logits: {type(predicted_logits)}")

  if isinstance(predicted_logits, tuple):

        predicted_logits = predicted_logits[0]

  if not isinstance(predicted_logits, np.ndarray):

        predicted_logits = np.array(predicted_logits)

  if len(predicted_logits.shape) == 3:

        predicted_logits = predicted_logits[:, 0, :]

  predicted_probabilities = expit(predicted_logits)

  threshold = 0.5

  predicted_classes = (predicted_probabilities >= threshold).astype(int)

  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average=None)

  accuracy = accuracy_score(y_valid, predicted_classes)

  return precision, recall, f1,accuracy

In [17]:
MAX_LENGTH = 256

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

n_splits = 5

mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)

In [18]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [19]:
models_list=models_list_1

models_name=models_name_1

x=models_list[0]

In [20]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [21]:
USE_LORA=True

In [41]:


torch.cuda.empty_cache()



print(x)

tokenizer = AutoTokenizer.from_pretrained(x)

model = AutoModelForSequenceClassification.from_pretrained(x, num_labels=5, ignore_mismatched_sizes=True,

                                                            quantization_config=quantization_config if USE_LORA else None,

                                                            low_cpu_mem_usage=True,

                                                          #  torch_dtype=torch.float16,
                                                           problem_type = "multi_label_classification",



                                                            )

unsloth/Llama-3.2-1B-Instruct


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()
    
    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]
            
            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(model)

['o_proj', 'q_proj', 'gate_proj', 'up_proj', 'k_proj', 'v_proj', 'down_proj']

In [26]:
a=

SyntaxError: invalid syntax (62411553.py, line 1)

In [43]:
model.train() # model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  model = prepare_model_for_kbit_training(model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  model = get_peft_model(model, config)



  # trainable parameter count

  model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

trainable params: 1,189,888 || all params: 1,237,014,528 || trainable%: 0.0962


In [44]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if model.config.pad_token_id is None:
        print("setting model pad token")
        model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

setting model pad token


'<|finetune_right_pad_id|>'

In [45]:


# model = model.to(device)

for param in model.parameters():

    param.data = param.data.contiguous()


os.environ['WANDB_DISABLED']="true"
training_args = train_arg()

train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=valid_dataset,

    compute_metrics=compute_metrics,

    # data_collator=data_collator

)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [46]:
checkpoints = sorted(glob.glob(os.path.join(trainer.args.output_dir, "checkpoint-*")), key=os.path.getmtime)



resume_checkpoint = checkpoints[-1] if checkpoints else None

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train(resume_from_checkpoint=resume_checkpoint)

model.config.use_cache = True


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
500,0.436900,0.413231,0.339202,0.429640,0.061014,0.093667
1000,0.385300,0.371876,0.386346,0.644115,0.158049,0.235514
1500,0.354600,0.350596,0.426839,0.666275,0.255820,0.356987
2000,0.342000,0.344319,0.450117,0.727600,0.290125,0.386189
2500,0.334400,0.334827,0.462246,0.723792,0.315364,0.426534
3000,0.321700,0.332235,0.462441,0.721793,0.308993,0.425717
3500,0.318000,0.331171,0.463028,0.736162,0.305072,0.424000


Type of predicted_logits: <class 'numpy.ndarray'>


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Type of predicted_logits: <class 'numpy.ndarray'>


...

/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Type of predicted_logits: <class 'numpy.ndarray'>


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Type of predicted_logits: <class 'numpy.ndarray'>


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


In [47]:
model.config.use_cache = True
precision, recall, f1,accuracy=evaluate_func(trainer)
new_data = pd.DataFrame({

  "Models_name": [x],

  "Precision": [precision],

  "Recall": [recall],

  "F1_score": [f1],

  "Accuracy": [accuracy]

})



metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)



model_confusion_matrices = []

predicted_classes = trainer.predict(valid_dataset).predictions







if isinstance(predicted_classes, tuple) :

  predicted_classes = predicted_classes[0]

predicted_classes = np.array(predicted_classes)

# print(f"Type after conversion: {type(predicted_classes)}")



if len(predicted_classes.shape) > 2:

  predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)

  # print(f"Shape after reshaping: {predicted_classes.shape}")









predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)



y_valid_np = np.array(y_valid)



for j in range(len(label_columns)):

    print(f"Computing confusion matrix for label: {label_columns[j]}")

    cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])



    # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])

    model_confusion_matrices.append(cm)

confusion_matrices.append(model_confusion_matrices)





print(f"{x} evaluates:")

for k, label in enumerate(label_columns):

  print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>


/tmp/ipykernel_30/1985767954.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)


Type of predicted_logits: <class 'numpy.ndarray'>
Computing confusion matrix for label: anger
Computing confusion matrix for label: fear
Computing confusion matrix for label: joy
Computing confusion matrix for label: sadness
Computing confusion matrix for label: surprise
unsloth/Llama-3.2-1B-Instruct evaluates:
Class 'anger': Precision: 0.6931, Recall: 0.2561, F1: 0.3740
Class 'fear': Precision: 0.8399, Recall: 0.4854, F1: 0.6152
Class 'joy': Precision: 0.7244, Recall: 0.3707, F1: 0.4904
Class 'sadness': Precision: 0.7096, Recall: 0.2213, F1: 0.3374
Class 'surprise': Precision: 0.7714, Recall: 0.1746, F1: 0.2847
Overall accuracy: 0.4630
-------------------------------------------------------------------------

